In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
from census import Census
from pprint import pprint
# import gmaps

# Google API Key
from config import gkey




In [2]:
# geocoordinates

target_coordinates = "33.753746, -84.386330"
# target_coordinates = "43.6187102, -116.2146068"
target_search = "mexican"
# meters is the units
target_radius = 1000
target_type = "restaurant"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": gkey
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

frame_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=input=location=33.753746%2C+-84.386330&keyword=mexican&radius=1000&type=restaurant&key=API_KEY_HERE"  

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [3]:
# print the response url, avoid doing for public github repos in order to avoid exposing key
print(response.url)

https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=33.753746%2C+-84.386330&keyword=mexican&radius=1000&type=restaurant&key=


In [4]:
# convert response to json
places_data = response.json()
    
# Print the json (pretty printed)
print(json.dumps(places_data, indent=4, sort_keys=True))
# places_data_json = pd.read_json('base_url')

{
    "error_message": "You must use an API key to authenticate each request to Google Maps Platform APIs. For additional information, please refer to http://g.co/dev/maps-no-account",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}


In [18]:
import requests

url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=Museum%20of%20Contemporary%20Art%20Australia&inputtype=textquery&fields=formatted_address%2Cname%2Crating%2Copening_hours%2Cgeometry&key=API_KEY_HERE"

payload={}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)

{
   "candidates" : [
      {
         "formatted_address" : "140 George St, The Rocks NSW 2000, Australia",
         "geometry" : {
            "location" : {
               "lat" : -33.8599358,
               "lng" : 151.2090295
            },
            "viewport" : {
               "northeast" : {
                  "lat" : -33.85824377010728,
                  "lng" : 151.2104386798927
               },
               "southwest" : {
                  "lat" : -33.86094342989272,
                  "lng" : 151.2077390201073
               }
            }
         },
         "name" : "Museum of Contemporary Art Australia",
         "opening_hours" : {
            "open_now" : false
         },
         "rating" : 4.4
      }
   ],
   "status" : "OK"
}



In [5]:
# Print the name and address of the first restaurant that appears
# print(places_data["results"][0]["name"]),(places_data["results"][0]["vicinity"])
# print(places_data["results"][0]["vicinity"])
# print(places_data["results"][10]["name"])
# print(places_data["results"][10]["vicinity"])


# name = []
# rating = []
# user_ratings_total = []
# latitude = []
# longatiude = []
# business_satus = [] 
# place_id = []








In [11]:
# df_nested_list = pd.json_normalize(base_url, record_path =['name'])


# df = pd.read_json(base_url)
# df.head()


# build our data frame columns
name = []
rating = []
user_ratings_total = []
lat = []
lng = []
business_status = []
price_level = []

# Loop through the list of cities and perform a request for data on each
# for i, row in places_data.iterrows():
# # for restaurnts in places_data:
#     response = requests.get(base_url, params=params).json()
    
    
    
    name.append(response["results"][i]["name"])
#     rating.append(response["results"][i]["rating"])
#     user_ratings_total.append(response["results"][i]["user_ratings_total"])
#     lat.append(response["results"][i]["geometry"]["location"]["lat"])
#     lng.append(response["results"][i]["geometry"]["location"]["lng"])
#     business_status.append(response["results"][i]["business_status"])
#     price_level.append(response["results"][i]["price_level"])

#     lat.append(response['coord']['lat'])
#     temp.append(response['main']['temp'])
print(name)

AttributeError: 'dict' object has no attribute 'iterrows'

In [ ]:
# atl_df=pd.DataFrame({"places_data":places_data})
# atl_df["place_id"] = ""
# atl_df["name"] = ""
# atl_df["rating"] = ""
# atl_df["user_ratings_total"] = ""
# atl_df["lat"] = ""
# atl_df["lng"] = ""
# atl_df["business_status"] = ""
# create a data frame from cities, lat, and temp
# weather_dict = {
#     "city": cities,
#     "lat": lat,
#     "temp": temp
# }
# weather_data = pd.DataFrame(weather_dict)
# weather_data.head()



# atl_dict = {
# #     "Place ID":place_id,
#     "Name":name,
#     "Rating":rating,
#     "Date":dates,
#     "User Total Rating":user_ratings_total,
#     "Latitude":lat,
#     "Longatiude":lng,
#     "Business Status":business_status,
#     "Price Level":price_level,


places_data_df=pd.DataFrame(places_data)
places_data_df.head()


In [ ]:
url="https://api.yelp.com/v3/businesses/search?term=delis&latitude=37.786882&longitude=-122.399972"
# Make request and store response
response = requests.get(url)

# Verify status code
response.status_code